In [4]:
%load_ext autoreload

In [436]:
%autoreload 2

from datetime import datetime as dt
from itertools import product
import os, sys

import matplotlib.pyplot as plt
import numpy as np
from plotly import graph_objects as go
from scipy.stats import norm
from sympy import lambdify, sympify
from vegas import AdaptiveMap, Integrator

# Local scripts found in 'scripts' directory, add to PYTHONPATH to import
mc_path = os.path.abspath(os.path.join('scripts'))
if mc_path not in sys.path:
    sys.path.append(mc_path)
from monte_carlo import *

## Variance

## $$\text{Var}\big(X\big)=E\big[X^2\big]+E\big[X\big]^2\text{ and }\text{Var}\big(E[X]\big)=\frac{\text{Var}\big(X\big)}{N}$$

# MC method:

### $$\displaystyle E[f]\equiv E_U[f]=\int_a^bf(x)\ \text{d}x=(b-a)\int_a^b\frac{f(x)}{b-a}\ \text{d}x\approx\frac{b-a}{N}\sum_{i=1}^Nf(x_i)\text{ where }x_i\sim U[a,b].$$

---

# Control Variate:

### $$\displaystyle\int_a^bf(x)\ \text{d}x\approx\frac{b-a}{N}\sum_{i=1}^N\left(f(x_i)+cg(x_i)\right)-cE[g(X)]\text{ where }x_i\sim U[a,b]\text{ and }E[g(x)]=I_g\text{ is known}.$$

### $$\text{The variance of is }f\text{ minimized when }\displaystyle c=-\frac{\text{Cov}(f,g)}{\text{Var}(g)}=-\frac{E[fg]-E[f]E[g]}{E[g^2]-E[g]^2}=\frac{I_gE[f]-E[fg]}{E[g^2]-I_g^2}$$
### $$\text{The variance becomes: }\text{Var}\Big(f(X)+cg(X)\Big)=\left(1-\text{Corr}(f,g)^2\right)\text{Var}(f)\text{ where }\text{Corr}(f,g)=\frac{\text{Cov}(f,g)}{\sqrt{\text{Var}(f)\text{Var}(g)}}$$

---

# Antithetic variates

### $$\displaystyle\text{Let }f(X)=\frac{f(X_1)+f(X_2)}{2}\text{ where }x_1=\text{CDF}^{-1}(u_1)\sim X_1\text{ and }x_2=\text{CDF}^{-1}(1-u_1)\sim X_2\text{ where }u\sim U[0,1]$$
### then
### $$\text{Var}\big(f(X)\big)=\frac{1}{4}\bigg[\text{Var}\big(f(X_1)\big)+\text{Var}\big(f(X_2)\big)+2\text{Cov}\big(f(X_1),f(X_2)\big)\bigg]$$

Since $X_1$ and $X_2$ have a negative correlation, their covariance will always be negative and thus $X$ will have a smaller variance than if $X_1$ and $X_2$ were uncorrelated (as it is in normal MC).

---

# Wiki Example: $\displaystyle f(x)=\frac{1}{1+x}$ with $x\sim U[0,1]$

In [ ]:
func = "1 / (x + 1)"
cvfunc = "x + 1"
soln = 3/2
print(f"Exact solution: {np.log(2)}")

## Normal MC

In [ ]:
mcprint(MC(func, 0, 1, use_vegas=False), sigfigs=8)

## Control Variate MC

In [ ]:
mcprint(MCcv(func, cvfunc, soln, cN=int(1e6), N=int(1e6), use_vegas=False), sigfigs=8)

The correlation coefficient is between -1 and 1. We want to maximize it with the choice of the variate function. Looking at the correlation between $\displaystyle\frac{1}{1+x}$ and $x+1$ to see variance reduction:

In [ ]:
print(f"{100 * corr(func, cvfunc, expval2=soln)**2:.3f}% reduction in variance")

## Antithetic Variate MC

In [ ]:
mcprint(MCav(func), sigfigs=8)

This works if $\text{Cov}(Y_1,Y_2)<0$. Here $\displaystyle Y_1=\frac{1}{1+X}$ and $\displaystyle Y_1=\frac{1}{1+(1-X)}=\frac{1}{2-X}$. So,

In [ ]:
Y1 = str(sympify(func))
Y2 = str(sympify(Y1.replace('x', '(1-x)')))
covy1y2 = cov(Y1, Y2)
print(f"The covariance is {covy1y2:.3f}. It is {'less' if covy1y2 < 0 else 'greater'} than zero.")

---

## Combining Control and Antithetic Variates

In [ ]:
# MAKE THIS ACTUALLY WORK

---

# MC From Distribution With Known Inverse CDF

### $$\int_a^bf(x)g(x)\text{d}x=\frac{1}{N}\sum_{i=1}^Nf(x_i)\text{ where }x_i\sim g(X)$$

$g(X)$ is the PDF of $f(X)$. Samples can be drawn from that distribution using a uniform distribution. If $U[0,1]$ is a uniform distribution, then $g(X)=\text{CDF}_{g}^{-1}(U)$. So using the inverse CDF, we can make draws using a uniform distribution.

## Your Very Own PDF Checker!

In [ ]:
xi, xf = -5, 5

xs = np.linspace(xi, xf, 1000)
dist = 'normal'
kwargs = {'mu': 0, 'sigma': 2}

fig = go.Figure()
fig.add_trace(go.Histogram(x=inv_cdfs[dist](**kwargs, N=100_000), histnorm='probability density', name='Inv CDF'))
fig.add_trace(go.Scatter(x=xs, y=pdfs[dist](xs, **kwargs), mode='lines', name='Exact PDF'))
fig.update_xaxes(range=(xi, xf))
fig.update_layout(title=f"Distribution: {dist.title()}", height=800)
fig.show()

## And checking if the variance found from MC is the actual variance (using a normal distribution)

In [ ]:
def check_var(mc_func, Ntrials, Nxs, Nstds, mc_func_kwargs):
    # Get distributions for expectation value and variance
    valvars = np.array([mc_func(**mc_func_kwargs) for _ in range(Ntrials)])
    vals, varis = valvars[:, 0], valvars[:, 1]

    # Get mean values for expectation value and standard deviation
    avgval = np.mean(vals)
    avgstd = np.mean(np.sqrt(varis))
    fitval, fitstd = norm.fit(vals, loc=avgval, scale=avgstd)

    mc_color = 'red'
    fit_color = 'green'
    xs = np.linspace(avgval - Nstds*avgstd, avgval + Nstds*avgstd, Nxs)

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=vals, histnorm='probability density', name='MC Values'))
    fig.add_trace(go.Scatter(x=xs, y=norm.pdf(xs, loc=fitval, scale=fitstd), line=dict(color=fit_color), name='Fitted Gaussian'))
    fig.add_trace(go.Scatter(x=xs, y=norm.pdf(xs, loc=avgval, scale=avgstd), line=dict(color=mc_color), name='MC Gaussian'))
    fig.update_layout(
        height=800,
        xaxis_title='Estimation of expectation value'
    )
    fig.add_annotation(
        x=0.99, y=0.98,
        xref="paper", yref="paper",
        showarrow=False,
        text=f'Mean: {fitval:.6f} | St Dev: {fitstd:.6f}',
        font_size=20,
        bgcolor=fit_color,
        bordercolor='black',
        borderwidth=1
    )
    fig.add_annotation(
        x=0.99, y=0.90,
        xref="paper", yref="paper",
        showarrow=False,
        text=f'Mean: {avgval:.6f} | St Dev: {avgstd:.6f}',
        font_size=20,
        bgcolor=mc_color,
        bordercolor='black',
        borderwidth=1
    )
    fig.show()

In [ ]:
# e.g.
check_var(MCav, 500, 100000, 8, dict(func='1 / (x + 1)', N=1000))

---

# Vegas Basics

In [ ]:
import vegas 
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

integ = Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

- `nitn` is number of iterations. Vegas tries to flatten the integrand via a transformation and each iteration, it uses info from the previous to optimize the transformation.
- `neval` is the maximum number of evaluations per iteration. 
- The weight average is weighted by the inverse variance so the first few iterations (which are very inaccurate) add very little to the weight average.
- $Q$ called $p$-value is the probability that a larger $\chi^2$ could result from random Gaussian fluctuations.
- A small value, $Q<0.1$, says that the estimates of the integral from different iterations do not agree with each other within error.
    - So `neval` must be increased to trust the error estimates, i.e. `neval` isn't guarenteeing Gaussian behavior.\\
- Computing cost is roughly proportional to `nitn`$*$`neval`.
- Using too many iterations can be bad. Use no more than 10-20 iterations beyond where Vegas has fully adapted.
- Systematic error vanishes by at least 1/`neval`.
- Increasing `neval` will guarantee a decrease in both statistical and systematic uncertainties.
- Increasing `nitn` will gaurantee to eventually give the wrong answer.
    - Statistical error falls like sqrt(1/`nitn`).
    - Systematic error isn't affected by `nitn`.
    - So eventually systematic errors become significant.

### Sometimes it is useful to throw away early iterations

In [ ]:
integ = vegas.Integrator([[-2, 2], [0, 2], [0, 2], [0., 2]])
result = integ(f, nitn=10, neval=1000)
print(result.summary())
print(f"Result={result}, Q={result.Q:.3f}")

A $Q=0$ means our result is completely unreliable. Sometimes Vegas finds the peak and we get a reasonable answer but most of the time we don't. So we throw away the first few inaccurate iterations.

In [ ]:
# Adapt to f, discard results
integ(f, nitn=10, neval=10000)

# `integ` has adapted to f, keep results
result = integ(f, nitn=10, neval=10000)
print(result.summary())
print(f"Result={result}, Q={result.Q:.3f}")

Can use non-rectangular bounds by using `if/else` statements in function. That is, if the value is outside your bounds, then return `0`. Choice of bounds for the `Integrator` object is important otherwise Vegas will spend a lot of time where the integral is zero. This is more pronounced the higher the dimension is.
- `alpha` controls how quickly Vegas adapts. It defaults to 0.5. If there are persistent, large fluctuations in the size of the per-iteration errors, then `alpha` may need to be reduced.

### Batch Integration

In [ ]:
def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * 1013.2118364296088

@vegas.batchintegrand
def f_batch(x):
    dim = x.shape[1]
    norm = 1013.2118364296088 ** (dim / 4)
    dx2 = 0
    for d in range(4):
        dx2 += (x[:, d] - 0.5) ** 2
    return np.exp(-dx2 * 100.) * norm

integ = vegas.Integrator(4 * [[0, 1]])

In [ ]:
%timeit integ(f, nitn=10, neval=10_000)

In [ ]:
%timeit integ(f_batch, nitn=10, neval=10_000)

Note a 10x speedup. Batch integration can also be coded with a class:

In [ ]:
class f_batch(vegas.BatchIntegrand):
    def __init__(self, dim):
        self.dim = dim
        self.norm = 1013.2118364296088 ** (dim / 4.)

    def __call__(self, x):
        dx2 = 0.0
        for d in range(self.dim):
            dx2 += (x[:, d] - 0.5) ** 2
        return np.exp(-100. * dx2) * self.norm

f = f_batch(dim=4)
integ = vegas.Integrator(f.dim * [[0, 1]])

integ(f, nitn=10, neval=2e5)
result = integ(f, nitn=10, neval=2e5)

# Testing Grounds

For an integral
\begin{align}
    I=\int_a^bf(x)\ \text{d}x
\end{align}
rewrite it over a new variable $y$
\begin{align}
    I=\int_0^1J(y)f(x(y))\ \text{d}y\qquad\text{where}\qquad J(y)=\frac{\partial{x}}{\partial{y}}
\end{align}
is the Jacobian of the transformation. The Monte Carlo estimation is then
\begin{align}
    I\approx\frac{1}{N}\sum_{i=1}^NJ(y_i)f(x(y_i))
\end{align}
where the $y_i$'s are drawn from a uniform distribution $U[0,1]$.

In [450]:
# My attempt using the docs for Adaptive map
def f(x):
    return 1 / (1 + x)

# `ninc` is the number of increments between 0 and 1
admap = AdaptiveMap([[0, 1]], ninc=100_000)

# Initial uniformly random choices
ny = 100
y = np.random.uniform(0, 1, (ny, 1))

# Initializing arrays
x = np.empty(y.shape, float)
jac = np.empty(y.shape[0], float)
f2 = np.empty(y.shape[0], float)

# Do 100 iterations of adaptation
for itn in range(100):
    admap.map(y, x, jac)

    # Don't understand why it's squared...
    for j in range(ny):
        f2[j] = (jac[j] * f(x[j])) ** 2

    admap.add_training_data(y, f2)
    admap.adapt(alpha=0.5)

# Do the sampling, not very accurate
n = 10_000
f(np.random.choice(admap.extract_grid()[0], n)).sum() / n

0.6703166240981648

In [449]:
# Using Prasanth's code, you get the wrong answer only if you adapt first which I don't understand
integration_region = [[0, 1]]
def importance_sampling(f, sampling_map, neval, rng):
    y = rng.uniform(low=0.0, high=1.0, size=(neval, 1))
    x = sampling_map(y)

    return sampling_map.jac(y) * f(x)

rng = np.random.RandomState(1)
integrator = Integrator(integration_region)
# Adapting integrator
integrator(f, nitn=20, neval=1000)
weight_values = importance_sampling(
    f=f,
    sampling_map=integrator.map,
    neval=10_000,
    rng=rng
)
print(f"mean = {np.mean(weight_values)}")

mean = 0.7207939516487306


In [451]:
# Vegas's black box way
intr = Integrator([[0, 1]])
result = intr(f)
print(result.summary())
print(f"I = {result[0]}")

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.693115(39)    0.693115(39)        0.00     1.00
  2   0.693115(37)    0.693115(27)        0.00     0.99
  3   0.693110(38)    0.693113(22)        0.01     0.99
  4   0.693168(37)    0.693127(19)        0.55     0.65
  5   0.693105(36)    0.693123(17)        0.48     0.75
  6   0.693208(37)    0.693137(15)        1.28     0.27
  7   0.693173(37)    0.693142(14)        1.20     0.30
  8   0.693121(37)    0.693140(13)        1.07     0.38
  9   0.693070(38)    0.693132(12)        1.31     0.23
 10   0.693194(36)    0.693139(12)        1.45     0.16

I = 0.693139(12)
